# Second Dataset Processing 

This notebook was used to clean and export a second dataset. At the end we merged the two data sets together into one large one. 

In [1]:
# for second DF : 
import pandas as pd
import json
import difflib
from collections import defaultdict

df = pd.read_csv("merged_jobs.csv")
df.rename(columns={
    "title": "positions",
    "start_date": "start_dates",
    "end_date": "end_dates"
}, inplace=True)
df.drop(columns=["person_id"], inplace=True)
df.head()


,positions,start_dates,end_dates,skills
0,Database Administrator,01/2014,02/2017,"['Database administration', 'Database', 'Ms sq..."
1,Database Administrator,04/2017,Present,"['Database administration', 'Database', 'Ms sq..."
2,Database Administrator,12/2008,08/2011,"['sql server management studio', 'visual studi..."
3,Oracle Database Administrator,06/2014,06/2016,"['DATABASES', 'ORACLE (4 years)', 'ORACLE 10G'..."
4,Oracle Database Administrator,06/2016,Present,"['DATABASES', 'ORACLE (4 years)', 'ORACLE 10G'..."


In [2]:
import ast

def ensure_list(x):
    if pd.isna(x):
        return []
    if isinstance(x, list):
        return x
    try:
        return ast.literal_eval(str(x))
    except:
        return [str(x)]

for col in ["positions", "start_dates", "end_dates", "skills"]:
    df[col] = df[col].apply(ensure_list)


In [3]:
df.head()

,positions,start_dates,end_dates,skills
0,[Database Administrator],[01/2014],[02/2017],"[Database administration, Database, Ms sql ser..."
1,[Database Administrator],[04/2017],[Present],"[Database administration, Database, Ms sql ser..."
2,[Database Administrator],[12/2008],[08/2011],"[sql server management studio, visual studio, ..."
3,[Oracle Database Administrator],[06/2014],[06/2016],"[DATABASES, ORACLE (4 years), ORACLE 10G, SQL,..."
4,[Oracle Database Administrator],[06/2016],[Present],"[DATABASES, ORACLE (4 years), ORACLE 10G, SQL,..."


In [4]:
first_row = df.loc[0, "start_dates"]
print(first_row)           # see the value
print(type(first_row))     # see the type of the list itself

# If it’s a list, check the first element
if isinstance(first_row, list) and first_row:
    print(type(first_row[0]))

['01/2014']
<class 'list'>
<class 'str'>


In [5]:
print(df.loc[0])

positions                               [Database Administrator]
start_dates                                            [01/2014]
end_dates                                              [02/2017]
skills         [Database administration, Database, Ms sql ser...
Name: 0, dtype: object


In [6]:
# Step 1: Make sure each column is a list
for col in ["positions", "start_dates", "end_dates", "skills"]:
    df[col] = df[col].apply(lambda x: x if isinstance(x, list) else [x])

# Step 2: Make sure dates are strings
for col in ["start_dates", "end_dates"]:
    df[col] = df[col].apply(lambda lst: [str(x) for x in lst])


In [7]:
import re 
from datetime import datetime

current_year = datetime.now().year

def extract_year(text):
    if not text:
        return None
    text = str(text)
    text = re.sub(r'(?i)present', str(current_year), text)
    match = re.search(r"(19|20)\d{2}", text)
    if match:
        return int(match.group(0))
    return None

rows = []


for _, row in df.iterrows():
    positions = row["positions"]
    starts = row["start_dates"]
    ends = row["end_dates"]
    skills = row["skills"]

    for i, position in enumerate(positions):
        start = starts[i] if i < len(starts) else None
        end = ends[i] if i < len(ends) else None

        rows.append({
            "position": position,
            "start_year": extract_year(start),
            "end_year": extract_year(end),
            "skills": skills
        })

df_new_expanded = pd.DataFrame(rows)

df_new_expanded["start_year"] = df_new_expanded["start_year"].astype("Int64")
df_new_expanded["end_year"] = df_new_expanded["end_year"].astype("Int64")


In [8]:
df_new_expanded.head()

,position,start_year,end_year,skills
0,Database Administrator,2014,2017,"[Database administration, Database, Ms sql ser..."
1,Database Administrator,2017,2025,"[Database administration, Database, Ms sql ser..."
2,Database Administrator,2008,2011,"[sql server management studio, visual studio, ..."
3,Oracle Database Administrator,2014,2016,"[DATABASES, ORACLE (4 years), ORACLE 10G, SQL,..."
4,Oracle Database Administrator,2016,2025,"[DATABASES, ORACLE (4 years), ORACLE 10G, SQL,..."


In [9]:
import json

# Group by start_year
year_groups = df_new_expanded.groupby("start_year")

json_data = []

for year, group in year_groups:
    positions_list = []
    for _, row in group.iterrows():
        positions_list.append({
            "position": row["position"],
            "skills": row["skills"]
        })
    json_data.append({
        "year": int(year),
        "positions": positions_list
    })

# Optional: sort by year
json_data = sorted(json_data, key=lambda x: x["year"])


In [10]:
with open("career_by_year2.json", "w") as f:
    json.dump(json_data, f, indent=4)


In [12]:
import glob, json

merged = []
for file in glob.glob("data/*.json"):
    with open(file) as f:
        merged += json.load(f)

with open("merged.json", "w") as f:
    json.dump(merged, f, indent=4)


In [13]:
df_new_expanded.to_csv("cleaned_jobs2.csv", index=False)

In [14]:
import pandas as pd

# Load both
df1 = pd.read_csv("cleaned_jobs1.csv")
df2 = pd.read_csv("cleaned_jobs2.csv")

# Combine them
merged_df = pd.concat([df1, df2], ignore_index=True)

# Optional: drop duplicates if any
merged_df = merged_df.drop_duplicates()

# Save merged version
merged_df.to_csv("merged_jobs.csv", index=False)
